In [1]:
import cv2 
import imutils
import numpy as np
background  = None

In [2]:
def run_avg (image, aWeight):
    global background
    
    if background is None:
        background =  image.copy().astype("float")
        return
    
    cv2.accumulateWeighted(image, background, aWeight)

In [3]:
def segment(image, threshold=28):
    global background

    difference = cv2.absdiff(background.astype("uint8"), image)

    thresholded = cv2.threshold(difference, threshold, 255, cv2.THRESH_BINARY)[1]
    
    (contours, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return
    
    else:
        segmented = max(contours, key=cv2.contourArea)
        return (thresholded, segmented)

In [4]:
if __name__ == "__main__":
 
    aWeight = 1.0
    
    num_frames = 0

    Web_Camera = cv2.VideoCapture(0)

    top, right, bottom, left = 100, 400, 400, 600

    while(True):
        (grabbed, frame) = Web_Camera.read()

        frame = imutils.resize(frame, width=800)

        frame = cv2.flip(frame, 1)
        
        frame_0 = frame.copy()
        index =  -1
        thickness =1
        color = (0, 0, 255)

        (height, width) = frame.shape[:2]

        roi = frame[top:bottom, right:left]


        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        Blur = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 30:
            run_avg(gray, aWeight)
        else:

            hand = segment(Blur)

            if hand is not None:
        
                (thresholded, segmented) = hand

                cv2.drawContours(frame_0, [segmented + (right, top)], index, color, thickness)
                cv2.imshow("With_Threshold", thresholded)

        cv2.rectangle(frame_0, (left, top), (right, bottom), (0,0,255), 1)

        num_frames += 1

        cv2.imshow("Video_Capture ", frame_0)

        keypress = cv2.waitKey(1) & 0xFF


        if keypress == ord("q"):
            break

Web_Camera.release()
cv2.destroyAllWindows()